# Music Genre and Composer Classification Using Deep Learning

## Classical Music Time Series Model Comparison
## AAI-511 Team 6 Final Project
## Part I : Data Preprocessing & Feature Extraction

Team 6:  Philip Felizarta, Sarah Durrani, Geoffrey Fadera

University of San Diego, Applied Artificial Intelligence

Date:  August 12, 2024

GitHub Repository: https://github.com/PhilipFelizarta/Classical-Music-Time-Series-Model-Comparison?query=PhilipFelizarta

In [1]:
import os
import pretty_midi
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import zipfile
import pandas as pd
import warnings
import pickle
from sklearn.preprocessing import LabelBinarizer
import warnings
from visual_midi import Plotter, Preset
import matplotlib.pyplot as plt

RANDOM_STATE = 0
TRAIN_SIZE = 0.80
VAL_TEST_SPLIT = 0.5

## Supporting Functions

In [11]:
# Unzip file and all zipped files within all its subdirectory levels
# By visual inspection, even after unzipping the parent directory 'archive', composers' subdirectories still contain Zip files.
# The following function unzips a Zip file, and recursively unzips all Zip files within it.

def unzipFileTree(directory, file):
    
    if file.endswith('.zip'):
        zip_filepath = os.path.join(directory, file)
        zip_filename = os.path.splitext(os.path.basename(zip_filepath))[0]
        zip_targetDir = os.path.join(directory, zip_filename)
        
        try:
            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                zip_ref.extractall(zip_targetDir)
                #print(f"\tUnzipped File: {zip_filepath}")
                #print(f"\tUnzipped to folder: {zip_targetDir}")
        except zipfile.BadZipFile:
            print(f"Error: {zip_filepath} is a bad zip file")
    else:
        return

    for root, dirs, files in os.walk(zip_targetDir):
        for file in files:
            unzipFileTree(root, file)


# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Read midi files under each 'composer' folder
# Input: root directory, composer string array
# Return: [List of imported prettyMidi files, List of corresponding composer]

def read_Midis_Composer (directory, composers = []):
    
    midis_data = []
    composer_data = []
    filepath_data = []

    for composer in composers:
        composerDir = os.path.join(directory, composer)
        
        for dirpath, dirs, files in os.walk(composerDir):  
            for filename in files: 
                fname = os.path.join(dirpath,filename) 
                
                try:
                   # with warnings.catch_warnings():
                   # warnings.simplefilter("error")
                    pm = pretty_midi.PrettyMIDI(fname)

                except:
                    if (fname.endswith('.mid') or fname.endswith('.MID')): 
                        print(f"Error loading MIDI file (Skipping):", fname)
                    else : 
                        print(f"Warning: Non-midi file encountered (Skipping):", fname)
                    continue
                
                # make all extensions ''.mid'
                
                
                base_name, _ = os.path.splitext(fname)
                new_fname = base_name + ".mid"
                os.rename(fname, new_fname)
                
                
                midis_data = np.append(midis_data, pm)
                composer_data = np.append(composer_data, composer)
                filepath_data = np.append(filepath_data, new_fname)
    
    return midis_data, composer_data, filepath_data

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# define a function that converts a pretty_midi array into a 'bag of notes'
# Dimensionality is reduced as Time and Instrument components are now added as features for each note.

def convertMidisList_OP1 (midisDataList = []):
    
    resultMidisList = []
    for midisData in midisDataList:
        
        flatMidi = []
        for instrument in midisData.instruments:
            for note in instrument.notes:
                start = note.start
                end = note.end  
                pitch = note.pitch
                velocity = note.velocity

                combinedEntry = np.array([start, end, pitch, velocity, instrument.program])
                flatMidi.append(combinedEntry)

        resultMidisList.append(np.array(flatMidi))

    return resultMidisList

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# parses flattened pretty_midi data (OPTION 1) and
# returns list of notes simultaneously being played at input timestamp

def sample_FlatMidi(midiFlat=[], timestamp=0):
    
    startTime_col = 0
    endTime_col = 1

    rows = (midiFlat[:, startTime_col] < timestamp) & (midiFlat[:, endTime_col] >= timestamp)
    selected_rows = midiFlat[rows]

    return selected_rows   

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Converts a flattened midi file (from OPTION 1) into a time-sequence of 2-D vectors
# each vector is of shape (Max Number of Notes Sampled in this midi at any time instance) X 3.

def convertFlatMidi_OP2(midiFlatOP1=[], Fs = 10):
    
    start = 0
    stop = np.max(midiFlatOP1[:,1])
    step = 1/Fs
    timeline = np.arange(start, stop + step, step)
    resultMidi = []
    maxNotesOneSample = 0
    
    # sample the flattened Midi at a given timestamp. save notes played simultaneously at that time instance.
    # remove start and end time attributes.
    for timestamp in timeline:
        sampledNotes = sample_FlatMidi(midiFlatOP1, timestamp)
        
        if len(sampledNotes) > maxNotesOneSample:
            maxNotesOneSample = len(sampledNotes)
        
        resultMidi.append(sampledNotes[:,2:5])
    
    # pad samples so final array is of size (Max Num of simultaneous Notes played at one time instance) X 3
    for i in range(0, len(resultMidi)):
        sample = resultMidi[i]
        samplePadded = np.pad(sample,((0, maxNotesOneSample - sample.shape[0]), (0, 0)), mode = 'constant')
        resultMidi[i] = samplePadded
    
        
    return np.array(resultMidi), maxNotesOneSample

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Converts flattened Midi List from OP1 into time-series representation
# Each midi from the output of OP1 is sampled given Fs (number of samples per second)
# 

def convertFlatMidisList_OP2 (midisFlatList_OP1=[], Fs = 10):
    resultMidisList = []
    resultMidisListPadded = []
    maxNotesPerSampleEntireList = 0
    
    for index, flatMidi in enumerate(midisFlatList_OP1):
        
        # convert each flattened Midi (from OP1) into (?,3) x time vector.
        flatMidi_OP2, maxNotesPerSampleThisMidi = convertFlatMidi_OP2(flatMidi, Fs)
        
        # store max number of notes simultaneously playing at any time instance during sampling.
        if (maxNotesPerSampleThisMidi > maxNotesPerSampleEntireList ):
            maxNotesPerSampleEntireList = maxNotesPerSampleThisMidi
            #print("Num simultaneous notes now at: ", maxNotesPerSampleEntireList)
            #print("Encountered at input no: ", index)
        
        # append converted midi to final list
        resultMidisList.append(flatMidi_OP2)
     
    # At this point, return List is 4-D:
    # (NUMBER OF INPUT MIDIs) X (? NUMBER OF TIME SAMPLES) X (? Number of Sampled Notes per Midi) X 3 (Note)
    # pad third dimension with zeros to match max number of sampled notes
    
    for i in range(0, len(resultMidisList)):
        
        # (time samples) x (max number of simultaneous notes for this midi) x 3
        midi = resultMidisList[i]
        
        # pad zeros on the second dimension to match max number of simultaneous notes for entire list
        midiPadded = np.pad(midi,((0,0), (0, maxNotesPerSampleEntireList - midi.shape[1]), (0, 0)), mode = 'constant')

        resultMidisListPadded.append(np.array(midiPadded))

    #return resultMidisList resultMidisListPadded, maxNotesPerSampleEntireList
    return resultMidisListPadded, maxNotesPerSampleEntireList

## Step 1. Data Import, Clean-Up, Exploratory Analysis, Pre-processing

In [3]:
# (1.1) Place 'archive.zip' in the same folder as this ipynb file and unzip using unzipFileTree () function.
unzipFileTree('./', 'archive.zip')

In [5]:
# (1.2) Define root directory of midi files to be processed
# Define composers to analyze
# Import midi files into a list, and generate a list of corresponding composer labels using the read_Midis_Composer() function

ROOT = "./archive/midiclassics"
COMPOSERS = ['Bach', 'Beethoven', 'Chopin', 'Mozart']

midisData, composersData, midiFilePathsData = read_Midis_Composer(ROOT, COMPOSERS)
np.savez('Step1_midisData.npz', *midisData)
np.savez('Step1_composersData.npz', *composersData)
np.savez('Step1_midiFilePaths.npz', *midiFilePathsData)


/Users/geoffreyfadera/anaconda3/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error loading MIDI file (Skipping): ./archive/midiclassics/Beethoven/Anhang 14-3.mid
Error loading MIDI file (Skipping): ./archive/midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid


In [6]:
# (1.3) Observe shape of data, and composer distribution

print("midisData shape: ", midisData.shape)
print("composersData shape: ", composersData.shape)
print("midiFilePathsData shape: ", midiFilePathsData.shape)

pd.DataFrame(composersData).value_counts()

midisData shape:  (1635,)
composersData shape:  (1635,)
midiFilePathsData shape:  (1635,)


Bach         1024
Mozart        256
Beethoven     219
Chopin        136
Name: count, dtype: int64

**NOTE:** There is clear class imbalance with the 'Bach' label having at least 4 times as many samples as the rest of the classes.

In [7]:
# (1.4) Convert Labels (composersData) to Binary Representations

label_enc=[]
label_enc = LabelBinarizer()
composersDataBin = label_enc.fit_transform(composersData)
composersDataBin.shape

# save to file
with open('Step1_labelBinarizer.pkl', 'wb') as f:
    pickle.dump(label_enc, f)

np.savez('Step1_composersDataBin.npz', *composersDataBin)

## Step 2. Data Conversion to Suitable Formats

Various options are explored in converting the midi data to suitable formats for deep learning models.

**Option 1:** Flattened pretty midi Notes vectors "Bag of Notes"<br>
**Option 2:** Time-series of pretty midi Notes vector<br>
**Option 3:** Time-series of Features extracted from raw audio

### STEP 2.0: Background on Midi files and pretty_midi python Library. 

"MIDI files, short for Musical Instrument Digital Interface files, are a type of data file that contains instructions for musical performance rather than actual audio recordings. MIDI files are essentially a set of instructions that tell electronic instruments or software what notes to play, how long to play them, and how loudly". (Perplexity, 2024). The MIDI file format is standardized and can be found here: (https://midi.org/standard-midi-files)

Pretty_midi is a python library that parses a midi file and converts it into "a container for MIDI data in an easily-manipulable format." (https://craffel.github.io/pretty-midi/#pretty-midi-prettymidi). The container is a list of instruments, with each instrument containing a list of notes to be played.

For each **Instrument**, the most basic attributes are: 
> **program:** provides the numerical mapping to actual musical instrument.(https://fmslogo.sourceforge.io/manual/midi-instrument.html). <br>
> **notes:** provides a sequence of 'Notes' to be played by the instrument.<br>
> Each of these notes is a vector containing {start, end, pitch, and velocity} values (https://craffel.github.io/pretty-midi/#pretty_midi.Note).
>> **start:** absolute time in seconds when this note is to be played <br>
>> **end:** absolute time in seconds when playing of this note ends.<br>
>> **pitch:** integer representation of actual musical notes (values 0-127)<br>
>> **velocity:** integer representation of the loudness of this note (values 0-127)

Effectively, a pretty_midi file is a 3-D vector of the following shape:
>(num of instruments) x (num of 'Notes' per instrument) x [start, end, pitch, velocity]

In [8]:
# For example, let us visualize one pretty_midi data
# The following only works for instrument program no 0 (piano)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    preset = Preset(plot_width=900)
    plotter = Plotter(preset, plot_max_length_bar=4)
    
    plotter.show_notebook(midisData[566])
    

Loading BokehJS ...

The plot above is a visualization of the pretty Midi data 'midisData[566]' (the shortest in time length). 
(Unfortunately, the visual_midi Plotter() function only works for one instrument program = 0)

It shows all the different "Notes" to be played across time. 
Each 'Note' is color coded and has a fixed time interval. 
Hovering the mouse over one 'Note' reveals a vector containing {program, pitch, velocity, duration, start_time, end_time}


### STEP 2.1 (OPTION 1): "BAG of NOTES" Conversion

Each midi file is converted into a collection of Note vectors where each vector contains [start, end, pitch, velocity, program]

Each imported midi file is converted:

From: 
> **shape:** (?, ?, 4) <br>
> **description:** (*num of instruments*) **X** (*num of Notes per instrument*) **X** (*NOTE vector:[start, end, pitch, velocity]*)

To:
> **shape:** (?, 5) <br>
> **description:** (*Number of new Note vectors*) **X** (*new NOTE vector:[start, end, pitch, velocity, program]*)

This flattens the pretty_midi vector along the 'instrument' dimension.<br>
Also, the Time component is part of each Note vector - i.e. 'start', 'end'.
The values are going to be directly used as features.


In [9]:
# convert each pretty_midi data into a (None, 5) vector using convertMidisList_OP1() function
midisDataOP1 = convertMidisList_OP1(midisData)

# Save the list of arrays to a .npz file
np.savez('Step2_midisOP1.npz', *midisDataOP1)

**OPTION 1 SUMMARY:** midisDataOP1 is of shape (1635, ?, 5)
> **1635 :** total num midis <br>
> **? :** number of unique Note vectors per midi file. value not fixed. <br>
> **5 :** Note vector (start, end, pitch, velocity, instrument)



### STEP 2.2 (OPTION 2): Time-series representation.

#### Each midi file is converted into a vector (TIME INSTANCES) X (LIST OF NOTES TO BE PLAYED GIVEN TIME INSTANCE) X (NOTE Vector)

**{TIME INSTANCES}** entire time length of a midi file divided into time instances of uniform interval defined by 1/Fs

**{LIST OF NOTES TO BE PLAYED GIVEN TIME INSTANCE}** shape varies depending on Fs, midi file, etc. To maintain uniform shape along this dimension, this list is padded with zeros to match the maximum number of notes sampled at any given time across all midis.

Each **{NOTE Vector}** contains (pitch, velocity, instrument).  'start' and 'end' attributes are removed from OP1.

In [10]:
# Define Fs, number of samples per second
# convert flattened midis from OP1 to time-series representation

Fs = 10
midisDataOP2, maxNotesAll = convertFlatMidisList_OP2(midisDataOP1, Fs)

Num simultaneous notes now at:  6
Encountered at input no:  0
Num simultaneous notes now at:  11
Encountered at input no:  4
Num simultaneous notes now at:  27
Encountered at input no:  32
Num simultaneous notes now at:  33
Encountered at input no:  85
Num simultaneous notes now at:  36
Encountered at input no:  958


In [12]:
# Save the list of arrays to a .npz file
np.savez('Step2_midisOP2_Fs10.npz', *midisDataOP2)

In [13]:
# 2 sample per second
midisDataOP2_Fs2, maxNotesAll = convertFlatMidisList_OP2(midisDataOP1, 2)
np.savez('Step2_midisOP2_Fs2.npz', *midisDataOP2_Fs2)

In [14]:
# 5 sample per second
midisDataOP2_Fs5, maxNotesAll = convertFlatMidisList_OP2(midisDataOP1, 5)
np.savez('Step2_midisOP2_Fs5.npz', *midisDataOP2_Fs5)

**OPTION 2 SUMMARY:** midisDataOP2 is of shape (1635, ?, 36, 3)
> **1635 :** total num midis <br>
> **? :** evenly spaced time instances per midi. The length varies depending on Fs and the time length of the midi file  <br>
> **36 :** notes simultaneously played per time instance <br>
> **3 :** Note vector (pitch, velocity, instrument)

### STEP 2.3 (OPTION 3): Audio Conversion then Feature Extraction

In [15]:
import os
import librosa
import numpy as np
from midi2audio import FluidSynth
import warnings

# Suppress specific warning from librosa
warnings.filterwarnings("ignore", category=UserWarning, module='librosa.core.pitch')

# Adjust the dataset path
dataset_dir = './content/Composer_Dataset/Composer_Dataset/NN_midi_files_extended'
output_wav_dir = './content/wav_files'
os.makedirs(output_wav_dir, exist_ok=True)


# Function to convert .mid to .wav (raw audio), using FluidR3_GM config, 
# Convert each Midi File to RAW Audio
def convert_midis_to_wavs(midiFilePaths, output_dir):
    
    wav_filePaths = []
    
    for i, midiFilePath in enumerate(midiFilePaths):
        # if .wav already exists for this .mid file, do not generate .wav file again
        wav_file = os.path.join(output_dir, os.path.basename(midiFilePath).replace('.mid', '.wav'))
        if os.path.exists(wav_file):
            print("File No: ", i,": audio already rendered. Skipping")
            wav_filePaths.append(wav_file)
            continue
        try:
            print("File No: ", i,": audio rendering")
            wav_filePaths.append(wav_file)
            fs = FluidSynth('/opt/homebrew/Cellar/fluid-synth/2.3.6/share/fluid-synth/sf2/FluidR3_GM.sf2')
            fs.midi_to_audio(midiFilePath, wav_file)
            
        except Exception as e:
            print(f"Error converting {midi_file} to WAV: {e}")
            return None
        
    return wav_filePaths


# Updated extract_audio_features function
def extract_audio_features(audio_file):
    try:
        y, sr = librosa.load(audio_file, sr=None)
        feature_vector = []
        
        # Mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        log_mel_spectrogram_flat = log_mel_spectrogram.flatten()
        
        # Chroma features
        if np.any(y):  # Ensure the audio is not silent
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            chroma_stft_flat = chroma_stft.flatten()
        else:
            chroma_stft_flat = np.zeros((12 * log_mel_spectrogram.shape[1],))

        # Tempo
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        if len(onset_env) > 0:
            tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
            if isinstance(tempo, np.ndarray):  # Check if tempo is an array
                tempo = float(tempo[0])  # Extract the first element as a scalar
        else:
            tempo = 0.0

        feature_vector = [len(y), sr,
                          np.array(log_mel_spectrogram), 
                          np.array(log_mel_spectrogram_flat),
                          np.array(chroma_stft),
                          np.array(chroma_stft_flat),
                          [tempo]
                     
                         ]
        
        return feature_vector

    except Exception as e:
        print(f"Error processing {audio_file}: {e}")
        return None



In [16]:
# Convert to RAW Audio using FLuidSynth

output_wav_dir = './content/wav_files'
audio_filesPath = convert_midis_to_wavs(midiFilePathsData, output_wav_dir)

File No:  0 : audio already rendered. Skipping
File No:  1 : audio already rendered. Skipping
File No:  2 : audio already rendered. Skipping
File No:  3 : audio already rendered. Skipping
File No:  4 : audio already rendered. Skipping
File No:  5 : audio already rendered. Skipping
File No:  6 : audio already rendered. Skipping
File No:  7 : audio already rendered. Skipping
File No:  8 : audio already rendered. Skipping
File No:  9 : audio already rendered. Skipping
File No:  10 : audio already rendered. Skipping
File No:  11 : audio already rendered. Skipping
File No:  12 : audio already rendered. Skipping
File No:  13 : audio already rendered. Skipping
File No:  14 : audio already rendered. Skipping
File No:  15 : audio already rendered. Skipping
File No:  16 : audio already rendered. Skipping
File No:  17 : audio already rendered. Skipping
File No:  18 : audio already rendered. Skipping
File No:  19 : audio already rendered. Skipping
File No:  20 : audio already rendered. Skipping
Fi

In [18]:
subdirFlat = "./AUDIO_FEATURES/Flat"
os.makedirs(subdirFlat, exist_ok=True)

subdirTimeSeries = './AUDIO_FEATURES/TimeSeries/'
os.makedirs(subdirTimeSeries, exist_ok=True)

IS_DEBUG = False

def extractAudioFeatures_andSave(wavFilePaths, isResultTimeSeries = True ):
    features_flat = []
    features_time_series = []
   
    for i, wavFile in enumerate(wavFilePaths):
        print("wavFile ", i, ":", wavFile)
        
        if wavFile:
            feature_vector = extract_audio_features(wavFile)
            
            if(IS_DEBUG):
                print("\tTime Series Length: ", feature_vector[0])
                print("\tSampling Rate: ", feature_vector[1])
                print("\tMel Spectrogram TimeSeries Shape: ", feature_vector[2].shape)
                print("\tMel Spectrogram Flat Shape: ", len(feature_vector[3]))
                print("\tChroma TimeSeries Shape: ", feature_vector[4].shape)
                print("\tChroma Flat Shape: ", len(feature_vector[5]))
                print("\tTempoShape: ", len(feature_vector[6]))
            
            if(isResultTimeSeries==False):
                # Flat Vector
                # Flatten features and concatenate, save
                feature_vector_flat = np.concatenate(([feature_vector[0]], # time length
                                                      [feature_vector[1]], # sampling rate
                                                      feature_vector[3], # flattened mel
                                                      feature_vector[5], # flattened chroma
                                                      feature_vector[6])) # tempo
                
           
                audioFeaturesPath = os.path.join(subdirFlat, os.path.basename(wavFile).replace('.wav', '.npz'))
            
                np.savez(audioFeaturesPath, *feature_vector_flat)
                
                
            
            else:
                # Time Series Audio Features
                feature_vector_time_series = np.concatenate((feature_vector[2], feature_vector[4]), axis=0) 
            
                audioFeaturesPath = os.path.join(subdirTimeSeries,'Step2.3_AUDIO_FEATURES_TIMESERIES_AUDIO_ID_{}.npz'.format(i))
                #print("Audio Features Path: ", audioFeaturesPath)
                
                np.savez(audioFeaturesPath, *feature_vector_time_series)
    
    return None 

In [ ]:
# Extract Audio Features in Time-Series, and save to npz
extractAudioFeatures_andSave(audio_filesPath, isResultTimeSeries = True)


In [ ]:
# For reference: Loading of npz example:

#loaded = np.load('Step2.3_audio_features_TimeSeries.npz')
#audio_features_time_series2 = [loaded[key] for key in loaded]

## STEP 2. FILES SUMMARY

Labels: "./Step1_composersDataBin.npz" <br>
LabelBinarizer: "./Step1_labelBinarizer.pkl" <br>

Data Conversion Op1: "./Step2_midisOP1.npz"<br>
Data Conversion Op2: "./Step2_midisOP2_Fs2.npz" or "./Step2_midisOP2_Fs5.npz" or "./Step2_midisOP2_Fs10.npz" <br>
Data Conversion Op3: "./AUDIO_FEATURES/TimeSeries/Step2.3_AUDIO_FEATURES_TIMESERIES_AUDIO_ID_X.npz"
